# Quick loop on twin color pairs  with multi colors G-R

- author Sylvie Dagoret-Campagne
- creation date 2024-07-30
- affiliation : IJCLab
- Kernel **w_2024_16**
- 
- update : 2024-08-03 : time_cut

** Run LoopTwinSource before this**
Must do quick run


In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146' # test this one today 2024/06/22

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
# Max angular distance betwwen stars of the pair
sep_max = 200.
sep_time = 1800.

## Definition of functions used

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def closestvisit(df1,df2):
    """
    This function associate pairs of closest observation for photmetry

    input : the 2 dataframes (first to one for which one want to find the closest obs in dtaframe 2)
    output : the dataframe of similar format as the first one having columns on nightobs , visit time, dt from second dataframe)
    """
    d1 =df1.copy(deep=True)
    d2 =df2.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["nightObs2","visit2","time2","deltat"])
    for key in d1.index:

        nightObs = key[0]
        time1 = d1.loc[key]["time"]
        d2["dt"] = np.abs(d2["time"]-time1)
        d2["Dt"] = d2["time"]-time1
        cut_sel = (d2["dt"] == d2["dt"].min()) #& (d2["nightObs"] == nightObs)
        sample = d2[cut_sel]
        #df.iloc[key] = [sample.index[0][0],sample.index[0][1],sample["time"].values[0],sample['dt'].values[0]]
        df.loc[key]["nightObs2"] = sample.index[0][0]
        df.loc[key]["visit2"] = sample.index[0][1]
        df.loc[key]["time2"] = sample["time"].values[0]
        df.loc[key]["deltat"] = pd.to_timedelta(sample["Dt"].values[0], unit="s").total_seconds()
        all_samples.append(sample)

    # select observations during the same night
    df_back = df.copy(deep=True)
    flag_sel = []
    for key in df_back.index:
        nightObs = key[0]
        nightObs2 = df_back.loc[key]["nightObs2"]
        flag_sel.append(nightObs == nightObs2)
    df = df[flag_sel]
    return df

In [ ]:
def getcutoutandimage(visitId,df_selectedvisits):
    """
    visitId : the calexp where are the two stars
    df_selectedvisits : the list of all sources and visit of the main targeted star
    """

    row_source = df_selectedvisits[df_selectedvisits.visit == visitId].iloc[0]  
    band = row_source['band']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)   
    src_cat = butler.get('sourceTable',**dataId, collections=collections)

    #select on star
    #src_cat = src_cat[(src_cat["extendedness"]==0) & (src_cat["parentSourceId"] == 0) ]
    #src_cat = src_cat[src_cat["extendedness"]==0 ]

    x1 = row_source['x_x']
    y1 = row_source['y_x']
    ra1= row_source['ra_x']
    dec1= row_source['dec_x']
    psfMag1  = row_source['psfMag_x']  
    psfSigma1 = row_source['psfSigma_x']
    psfMagDiffMmag1 = row_source['psfMagDiffMmag_x'] 
    apFlux_50_0_instFlux1 = row_source['apFlux_50_0_instFlux_x']
    apFlux_50_0_instFluxErr1 = row_source['apFlux_50_0_instFluxErr_x']


    x2 = row_source['x_y']
    y2 = row_source['y_y']
    ra2= row_source['ra_y']
    dec2= row_source['dec_y']
    psfMag2  = row_source['psfMag_y']  
    psfSigma2 = row_source['psfSigma_y']
    psfMagDiffMmag2 = row_source['psfMagDiffMmag_y'] 
    apFlux_50_0_instFlux2 = row_source['apFlux_50_0_instFlux_y']
    apFlux_50_0_instFluxErr2 = row_source['apFlux_50_0_instFluxErr_y']

    
    # searcch the neighbouring stars
    #separation = find_neighbourg(ra_target,dec_target,src_cat)
    #src_cat["sep"] = separation


    spherePointCenter = lsst.geom.SpherePoint(ra1*lsst.geom.degrees, dec1*lsst.geom.degrees)
    spt = lsst.geom.SpherePoint(ra2*lsst.geom.degrees, dec2*lsst.geom.degrees)
    ang = spherePointCenter.separation(spt)
    ang_arcsec = ang.asArcseconds()
    
   
    x0 = (x1+x2)/2.
    y0 = (y1+y2)/2.
    ra0 = (ra1+ra2)/2.
    dec0 = (dec1+dec2)/2.
    

    dx1 = x1-x0
    dy1 = y1-y0
    dx2 = x2-x0
    dy2 = y2-y0
    
    
    boxSize = int(max(np.abs(x1-x2),np.abs(y1-y2)))
    xmin= x0-boxSize
    xmax= x0+boxSize
    ymin= y0-boxSize
    ymax= y0+boxSize
        
    
    minBbox = geom.Point2I(int(x0) - boxSize ,int(y0) - boxSize)
    maxBbox = geom.Point2I(int(x0) + boxSize, int(y0) + boxSize)
    
    srcBbox = geom.Box2I(minBbox, maxBbox)


    row_source['x0'] = x0
    row_source['y0'] = y0
    row_source['xmin'] = xmin
    row_source['ymin'] = ymin
    row_source['xmax'] = xmax
    row_source['ymax'] = ymax
    row_source['ra0'] = ra0
    row_source['dec0'] = dec0

    row_source['x1'] = x1
    row_source['y1'] = y1
    row_source['dx1'] = dx1
    row_source['dy1'] = dy1
    row_source['ra1'] = ra1
    row_source['dec1'] = dec1

    row_source['x2'] = x2
    row_source['y2'] = y2
    row_source['dx2'] = dx2
    row_source['dy2'] = dy2
    row_source['ra2'] = ra2
    row_source['dec2'] = dec2
    
    
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        #subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        #subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        cutout=subimg.image.array
        
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        cutout = None

    df_pair = row_source.to_frame()
    return row_source,cutout



In [ ]:
def GetFluxes2StarsTwoBands(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    May plot some calexp cutout for checking
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"])
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]
            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,
                                           airmassb1,airmassb2,x11,y11,x12,y12,x21,y21,x22,y22]
            entrycount+=1

        
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue
      

        if idx%10 == 0:
            if cutoutb1 is not None and cutoutb2 is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(row_pair_b1,cutoutb1,ax=ax1)
                plotcutout(row_pair_b2,cutoutb2,ax=ax2)
                plt.show()

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

     # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    
    
    return df_col_b1b2


In [ ]:
def GetFluxes2StarsTwoBandsNoCutout(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    NO NOT plot some calexp cutout contrary to above function
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"                                         
                                         ])
    
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            #row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            #row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 
            row_pair_b1  = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]  
            row_pair_b2  = df_selectedvisits_b2[df_selectedvisits_b2.visit == visit2].iloc[0]  

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 
            
            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]    
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]

            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,airmassb1,airmassb2,
                                           x11,y11,x12,y12,x21,y21,x22,y22]

            entrycount+=1
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # absolute relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

    # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    


    return df_col_b1b2

In [ ]:
def closesttimespectrophotom(df_photpair,df_spec):
    """
    Search the closest spectroscopic time for a photometric light curve

    - calculate with assign
    df_spec[["Time"]].assign( ddt = lambda row : np.abs((row["Time"].loc[0].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()))
    - calculate with apply two quantities : dt_abs, dt_rel    
    df_spec[["dt_abs","td_rel"]]=df_spec[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()),(row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()],axis=1,result_type='expand')
    """
    d1 =df_photpair.copy(deep=True)
    d2 =df_spec.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["time1","time2","deltat"])
    
    # loop on light curve datapoints times
    all_spectro_associated = []
    for key in d1.index:
        # take the time of the first source 
        # remember dt must be small
        tphotometric = df_photpair["time1"].loc[key]
        df_spec_copy = df_spec.copy(deep=True)
        # compute relative and absolute time difference of spectro wrt photmetry
        df_spec_copy[["dt_abs_specphot","td_rel_specphot"]]=df_spec_copy[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - tphotometric).total_seconds()),(row["Time"].replace(tzinfo=None) - tphotometric ).total_seconds()],axis=1,result_type='expand')
        # find the minimum on absolute time difference between spectro and photometry
        df_spec_sel = df_spec_copy[ df_spec_copy["dt_abs_specphot"] == df_spec_copy["dt_abs_specphot"].min()]
        all_spectro_associated.append(df_spec_sel)

    # merge all datapoint on single dataframe
    df_spec_sel_concat = pd.concat(all_spectro_associated)
    df_spec_sel_concat.reset_index(level=0, inplace=True)

    # set nan if more than 12*3600 seconds outside
    
    return df_spec_sel_concat


## Selected visits

### Spectroscopy

In [ ]:
atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
#df_spec["PWV [mm]_x"]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8))
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
df_spec.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)
plt.tight_layout()

## Photometry

#### read list of all possible pairs

In [ ]:
df_pair = pd.read_csv("all_pairs.csv",index_col=0)
df_pair.head()

In [ ]:
len(df_pair)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_pair["sep"].hist(bins=50,ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")


### Select the pairs

In [ ]:
#sep_max = 100.
df_pair = df_pair[df_pair["sep"]<sep_max]

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_pair["sep"].hist(bins=50,ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")

## Read pairs

In [ ]:
path_input = f"data_coloredlightcurvesG_R_thmax_{sep_max:.0f}"
path_input = f"data_coloredlightcurvesG_R_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}"
if not os.path.exists(path_input):
    print(f"Missing directory {path_input}  ==> Must run notebook LoopTwinSourcesMultiColorLightCurveToFitsfromPairs-Auxtel-ColorGR.ipynb")
    assert False

### Read summary

In [ ]:
#df_summary = pd.read_csv(os.path.join(path_input,f"photompairsummaryG_R_thmax_{sep_max:.0f}.csv"),index_col=0)
df_summary = pd.read_csv(os.path.join(path_input,f"photompairsummaryG_R_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"),index_col=0)

In [ ]:
df_summary

## Read all colors_diff

In [ ]:
NPairs = int(df_summary["idx_pair"].max())

In [ ]:
all_colors_diff = []
for idx in range(NPairs):
    num_str = str(idx).zfill(4)
    #input_filename = f"color_lightcurveG_R_{num_str}_thmax_{sep_max:.0f}.csv"
    input_filename = f"color_lightcurveG_R_{num_str}_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"
    input_fullfilename = os.path.join(path_input,input_filename)
    df_col = pd.read_csv(input_fullfilename,index_col=0)
    if len(df_col) > 0:
        # convert date string in datetime format for plotting date
        df_col["time1"] = df_col.apply( lambda row : pd.to_datetime(row["time1"]),axis=1,result_type="expand")
        df_col["time2"] = df_col.apply( lambda row : pd.to_datetime(row["time2"]),axis=1,result_type="expand")
        df_col["Time"] = df_col.apply( lambda row : pd.to_datetime(row["Time"]),axis=1,result_type="expand")
        df_col["DATE-OBS"] = df_col.apply( lambda row : pd.to_datetime(row["DATE-OBS"]),axis=1,result_type="expand")
       
    all_colors_diff.append(df_col)

In [ ]:
Npairs = len(all_colors_diff)

In [ ]:
props = dict(boxstyle='round', facecolor="white", alpha=1)
textstr = '\n'.join((
    r'$N_{pairs}=%.0f$' % (Npairs, ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

In [ ]:
all_colors_diff[0][["time1","time2","dt","DATE-OBS","Time","dt_abs_specphot","td_rel_specphot"]] 

### Check good/bad light curves

In [ ]:
pd.options.display.max_rows = 300
df_summary["drefcol_psf_ap"] = df_summary["refcol_psf_md"] - df_summary["refcol_ap_md"]
df_summary

In [ ]:
df_summary[df_summary.refcol_ap_md < 0.02]

In [ ]:
df_summary[df_summary.refcol_ap_md > 0.2]

### Plot all relative colors histograms

In [ ]:
cmap = mpl.colormaps['jet']
colors_forplots = cmap(np.linspace(0, 1, Npairs))

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(12,6))
all_objectplotcolor = [] # compute median color per object
all_Ntimepoints = []
for idx,df in enumerate(all_colors_diff):
    df["psfcol12_s12"].hist(bins=100,range=(0.,1.4),ax=ax1,histtype="step",color=colors_forplots[idx],lw=3)
ax1.set_title("relative color diff with psf flux")
ax1.text(0.8, 0.9, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

for idx,df in enumerate(all_colors_diff):
    df["apcol12_s12"].hist(bins=100,range=(0,1.4),ax=ax2,histtype="step",color=colors_forplots[idx],lw=3)
    data =  df["apcol12_s12"].values
    if len(data)>=1:
        data= data[np.isfinite(data)]
        all_objectplotcolor.append(np.median(data))
        all_Ntimepoints.append(len(data))
    else:
        all_objectplotcolor.append(0.)
        
ax1.set_xlabel("PSF : $|(G-R)_1 - (G-R)_2|$")
ax2.set_xlabel("AP35 : $|(G-R)_1 - (G-R)_2|$")

ax2.set_title("relative color diff with Ap35 flux")
ax2.text(0.8, 0.9, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
plt.tight_layout()

## Plots

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=1.4)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)


https://matplotlib.org/stable/users/explain/colors/colorbar_only.html

In [ ]:
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=1.4)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='$|(G_1-R_1)-(G_2-R_2)|$')


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):

        #print(idx,df_col)
        df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(G-R)_1-(G-R)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(G-R)_1-(G-R)_2|$")
ax2.set_ylim(0.,1.5)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.2
DCOLMAX = 0.3
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap

DCOLMIN = 0.8
DCOLMAX = 1.4

jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.5
DCOLMAX = 0.8
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(G-R)_1-(G-R)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(G-R)_1-(G-R)_2|$")
ax4.set_ylabel("PSF: $|(G-R)_1-(G-R)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry Ap35: $|(G-R)_1-(G-R)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(G-R)_1-(G-R)_2|$")
ax2.set_ylim(0.,1.4)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.8
DCOLMAX = 1.4
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap
DCOLMIN = 0.1
DCOLMAX = 0.2
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.5
DCOLMAX = 0.8
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry: $|(G-R)_1-(G-R)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(G-R)_1-(G-R)_2|$")
ax4.set_ylabel("Ap35: $|(G-R)_1-(G-R)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

## Check if spread varies with (G-R) relative difference

In [ ]:
df_summary

In [ ]:
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}){:}({:},{:})".format(index, int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))
    #print(text)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(18,30))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax1,s=80, c= "sep",cmap="jet", label="$|(G-R)_1 - (G-R)_2|$", grid=True,colorbar=True)
sc2=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax2,s=80, c= "nclip",cmap="jet", label="$|(G-R)_1 - (G-R)_2|$", grid=True,colorbar=True)
sc3=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax3,s=80, c= "magmax",cmap="jet", label="$|(G-R)_1 - (G-R)_2|$", grid=True,colorbar=True)

ax1.set_title(title)
ax1.set_title("$\\sigma (|(G-R)_1 - (G-R)_2|) vs \overline{(|(G-R)_1 - (G-R)_2|}$")
ax1.set_xlabel("$\overline{(|(G-R)_1 - (G-R)_2|}$ (mag)")
ax1.set_ylabel("$\\sigma (|(G-R)_1 - (G-R)_2|) $ (mag)")



ax2.set_title("$\\sigma (|(G-R)_1 - (G-R)_2|) vs \overline{(|(G-R)_1 - (G-R)_2|}$")
ax2.set_xlabel("$\overline{(|(G-R)_1 - (G-R)_2|}$ (mag)")
ax2.set_ylabel("$\\sigma (|(G-R)_1 - (G-R)_2|) $ (mag)")


ax3.set_title("$\\sigma (|(G-R)_1 - (G-R)_2|) vs \overline{(|(G-R)_1 - (G-R)_2|}$")
ax3.set_xlabel("$\overline{(|(G-R)_1 - (G-R)_2|}$ (mag)")
ax3.set_ylabel("$\\sigma (|(G-R)_1 - (G-R)_2|) $ (mag)")

#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1.ax.set_xlabel('angular separation (arcsec)',rotation=0)

#cbar2 = plt.colorbar(ax2.collections[0],ax=ax2,orientation = 'horizontal',shrink=0.6)
#cbar2.ax.set_xlabel('number of points',rotation=0)

for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["muclip"]
    y = df_summary.iloc[index]["sigmaclip"]
    
    if index%2 == 0:
        ax1.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')

plt.suptitle(suptitle)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,10))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(G-R)_1 - (G-R)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(G-R)_1 - (G-R)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (Z or Y) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(G-R)_1 - (G-R)_2|$ (mag)')

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,16))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(G-R)_1 - (G-R)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(G-R)_1 - (G-R)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (G or R) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(G-R)_1 - (G-R)_2|$ (mag)')
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["magmax"]
    y = df_summary.iloc[index]["nclip"]
    
    if index%2 == 0:
        ax1.text(x-0.1,y+2,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.1,y-2,text,size= 'large',horizontalalignment='center',verticalalignment='center')

## Select good pairs

In [ ]:
#df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.05))
df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<15.5) & (row["nclip"]>20) )

In [ ]:
df_summary_sel

In [ ]:
good_indexes = df_summary_sel[df_summary_sel["select"]]["idx_pair"].values.astype(int)
print("good_indexes ",good_indexes)
Ngood = len(good_indexes)

In [ ]:
df_summary["selection_good"] = df_summary.apply(lambda row : int(row["idx_pair"]) in good_indexes ,axis=1)

In [ ]:
df_summary[df_summary["selection_good"]]

In [ ]:
props = dict(boxstyle='round', facecolor="white", alpha=1)
textstr0 = '\n'.join((
    r'$N_{pairs}=%.0f$' % (len(good_indexes), ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=1.4)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if idx in good_indexes:
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0,grid=True,label="")
        
        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)


ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aper Photometry: $|(G-R)_1-(G-R)_2|$ (selected pairs on mag)")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Aperture: $|(G-R)_1-(G-R)_2|$")
ax2.set_ylim(0.,1.4)
ax2.text(0.01, 0.95, textstr0, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:

fig,ax  = plt.subplots(figsize=(6,6))


for idx_pair,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx_pair]

    if idx_pair in good_indexes:
        refcol_ap_md = (df_summary[df_summary["idx_pair"].astype(int) == idx_pair]["refcol_ap_md"]).values[0]
        #df_col.plot(x="PWV [mm]_x",y="apcol12_s12",ax=ax,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax.errorbar(df_col["PWV [mm]_x"],df_col.apcol12_s12-refcol_ap_md,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        
ax.grid()       
title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

## Loop to plot relative light curves one by one and DO straight line fitting

In [ ]:
def func_line(x, a, b):
    """
    straight line model
    """
    return a * x + b 
xfit = np.linspace(0.,10.,50)

In [ ]:
df_fit_ap = pd.DataFrame(columns=["idx_pair","idx_summary","color","slope","slope_err","sep"]) 

date_form = DateFormatter("%y-%m-%d")

count = 0
for good_idx in good_indexes:

    fig = plt.figure(figsize=(12,8),constrained_layout=True)
    gs = GridSpec(2, 2, wspace=0, hspace=0.01, height_ratios=[1,1],figure=fig)

    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0],sharex=ax1)
    ax3 = fig.add_subplot(gs[0,1])
    ax4 = fig.add_subplot(gs[1,1])

    df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="all PWV meas by spectro")


    # get the light curve
    df_col = all_colors_diff[good_idx]
    # Extract the row in df_summary with the pair number good_idx
    df_summary_row = df_summary[df_summary["idx_pair"].astype(int) == good_idx]

    # extract the corresponding index and refcol_ap_md column
    index_in_summary = df_summary_row.index[0]
    refcol_ap_md = df_summary_row["refcol_ap_md"].values[0]

    
    # compute a recentered colors fro plotting
    df_col["relapcol"] = df_col.apcol12_s12-refcol_ap_md

    separation_angle = df_summary.loc[index_in_summary,"sep"]
  
    # compute the string
    textstr = '\n'.join((
    "(G-R) diff star pair :"
    r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
    r'$|(G-R)_1- (G-R)_2| \simeq%.3f$  mag' % (df_summary.loc[index_in_summary,"refcol_ap_md"], )))

    title = "pair {} :: obj1= {} obj2 = {} ".format(good_idx,df_summary.loc[index_in_summary,"idx_obj1"].astype(int),
                                                    df_summary.loc[index_in_summary,"idx_obj2"].astype(int))
    ax1.set_title(title)
    ax1.set_ylabel("PWV [mm]_x (spectro)")
   
    
    ax2.errorbar(df_col.time1,df_col.apcol12_s12-refcol_ap_md,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
    df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0,grid=True,label="PWV meas within 30 min")
    #ax2.text(0.01, 0.97, textstr, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    ax2.set_ylabel("relative apert col diff (mag)")
    ax2.set_xlabel("time")

    ax3.errorbar(df_col["PWV [mm]_x"],df_col.relapcol,xerr=df_col["PWV [mm]_err_x"],yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt=".")
    df_col.plot.scatter(x="PWV [mm]_x",y="relapcol",c="airmassb1",marker= "o",s=50,ax=ax3,colormap="jet")
    ax3.grid()
    ax3.set_ylabel("relative apert col diff (mag)")
    ax3.set_title("fit line aperture color diff vs PWV")
    #ax4.errorbar(df_col["PWV [mm]_x"],df_col.relapcol,xerr=df_col["PWV [mm]_err_x"],yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
    #ax4.grid()
    #ax4.set_xlabel("PWV [mm]_x (spectro)")
    ax4.set_axis_off()

    idx_tofit = (~np.isnan(df_col["PWV [mm]_x"])) & (df_col["PWV [mm]_x"] >=1) & (df_col["PWV [mm]_x"] <=9) 
  
    X = df_col["PWV [mm]_x"][idx_tofit]
    Y = df_col.relapcol[idx_tofit]
    EY = df_col.apcol12Err_s12[idx_tofit]
    pearsonr = scipy.stats.pearsonr(X, Y) 
    spearmanr = scipy.stats.spearmanr(X, Y)
   
    output_fit =  curve_fit(func_line,X,Y,sigma=EY,p0= [0,0],absolute_sigma=True,method='lm',full_output=True)
    popt= output_fit[0]
    pcov = output_fit[1]
    perr = np.sqrt(np.diag(pcov))
    yfit = func_line(xfit,*popt)
    ax3.plot(xfit,yfit,'r-')
    textstr2 = '\n'.join((
    r'(G-R) diff star pair : %.0f' % (good_idx,),
    r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
    r'$|(G-R)_1- (G-R)_2| \simeq%.3f$  mag' % (df_summary.loc[index_in_summary,"refcol_ap_md"], ),
    r"Fit result :",
    r'slope = %.3f +/- %.3f mmag/mm' % (1000*popt[0],1000*perr[0],),
    r'pearson r = %.3f , p-val = %.5f' % (pearsonr[0],pearsonr[1],),))
    #ax4.plot(xfit,yfit,'r-')
    ax4.text(0.05, 0.8, textstr2, transform=ax4.transAxes, fontsize=14,verticalalignment='top', bbox=props)

    df_fit_ap.loc[count] = [good_idx, index_in_summary, refcol_ap_md, popt[0]*1000, perr[0]*1000, separation_angle]  
    
    tmin = df_col["time1"].min()
    tmax = df_col["time1"].max()
    dt = tmax-tmin
    tmin = tmin-dt/10
    tmax = tmax+dt/10

    the_tmin=tmin
    the_tmax=tmax

    ax2.set_xlim(the_tmin,the_tmax)
    ax2.grid()
    #plt.xticks(rotation=45)
    ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=45, ha='right',fontsize=10)
    plt.show()
    count += 1
    #if count > 2:
    #    break
   


## Plot slope vs color

In [ ]:
df_fit_ap 

In [ ]:
fig,ax  = plt.subplots(figsize=(10,6))
ax.errorbar(x=df_fit_ap["color"],y=df_fit_ap["slope"],yerr=df_fit_ap["slope_err"],ecolor="k",fmt=".",color="k") 
sc = df_fit_ap.plot.scatter(x="color",y="slope",c="sep",marker= "o",s=50,ax=ax,colormap="jet")
ax.grid()
#ax.set_ylim(-5.,10)
ax.set_title("Slope vs relative aperture color $\Delta C = |(G-R)_1- (G-R)_2|$")
ax.set_xlabel("relative color $\Delta C = |(G-R)_1- (G-R)_2|$ (mag) ")
ax.set_ylabel("$d(\Delta C)/d(PWV)$ (mmag/mm) ")
#cbar = plt.colorbar(sc)
#cbar.ax.set_xlabel('separation angle (arcsec)', rotation=0)

## PSF curves

In [ ]:
df_fit_psf = pd.DataFrame(columns=["idx_pair","idx_summary","color","slope","slope_err","sep"]) 

date_form = DateFormatter("%y-%m-%d")

count = 0
for good_idx in good_indexes:

    fig = plt.figure(figsize=(12,8),constrained_layout=True)
    gs = GridSpec(2, 2, wspace=0, hspace=0.01, height_ratios=[1,1],figure=fig)

    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0],sharex=ax1)
    ax3 = fig.add_subplot(gs[0,1])
    ax4 = fig.add_subplot(gs[1,1])

    df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="all PWV meas by spectro")


    # get the light curve
    df_col = all_colors_diff[good_idx]
    # Extract the row in df_summary with the pair number good_idx
    df_summary_row = df_summary[df_summary["idx_pair"].astype(int) == good_idx]

    # extract the corresponding index and refcol_ap_md column
    index_in_summary = df_summary_row.index[0]
    refcol_psf_md = df_summary_row["refcol_psf_md"].values[0]

    
    # compute a recentered colors fro plotting
    df_col["relpsfcol"] = df_col.psfcol12_s12-refcol_psf_md

    separation_angle = df_summary.loc[index_in_summary,"sep"]
  
    # compute the string
    textstr = '\n'.join((
    "(G-R) diff star pair :"
    r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
    r'$|(G-R)_1- (G-R)_2| \simeq%.3f$  mag' % (df_summary.loc[index_in_summary,"refcol_psf_md"], )))

    title = "pair {} :: obj1= {} obj2 = {} ".format(good_idx,df_summary.loc[index_in_summary,"idx_obj1"].astype(int),
                                                    df_summary.loc[index_in_summary,"idx_obj2"].astype(int))
    ax1.set_title(title)
    ax1.set_ylabel("PWV [mm]_x (spectro)")
   
    
    ax2.errorbar(df_col.time1,df_col.psfcol12_s12-refcol_psf_md,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
    df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0.5,grid=True,label="PWV meas within 1Hour")
    #ax2.text(0.01, 0.97, textstr, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    ax2.set_ylabel("relative psf col diff (mag)")
    ax2.set_xlabel("time")

    ax3.errorbar(df_col["PWV [mm]_x"],df_col.relpsfcol,xerr=df_col["PWV [mm]_err_x"],yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt=".")
    df_col.plot.scatter(x="PWV [mm]_x",y="relpsfcol",c="airmassb1",marker= "o",s=50,ax=ax3,colormap="jet")
    ax3.grid()
    ax3.set_ylabel("relative psf col diff (mag)")
    ax3.set_title("fit line psf color diff vs PWV")
    #ax4.errorbar(df_col["PWV [mm]_x"],df_col.relapcol,xerr=df_col["PWV [mm]_err_x"],yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
    #ax4.grid()
    #ax4.set_xlabel("PWV [mm]_x (spectro)")
    ax4.set_axis_off()

    idx_tofit = (~np.isnan(df_col["PWV [mm]_x"])) & (df_col["PWV [mm]_x"] >=1) & (df_col["PWV [mm]_x"] <=9) 
  
    X = df_col["PWV [mm]_x"][idx_tofit]
    Y = df_col.relpsfcol[idx_tofit]
    EY = df_col.psfcol12Err_s12[idx_tofit]
    pearsonr = scipy.stats.pearsonr(X, Y) 
    spearmanr = scipy.stats.spearmanr(X, Y)
   
    output_fit =  curve_fit(func_line,X,Y,sigma=EY,p0= [0,0],absolute_sigma=True,method='lm',full_output=True)
    popt= output_fit[0]
    pcov = output_fit[1]
    perr = np.sqrt(np.diag(pcov))
    yfit = func_line(xfit,*popt)
    ax3.plot(xfit,yfit,'r-')
    textstr2 = '\n'.join((
    r'(G-R) diff star pair : %.0f' % (good_idx,),
    r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
    r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
    r'$|(G-R)_1- (G-R)_2| \simeq%.3f$  mag' % (df_summary.loc[index_in_summary,"refcol_psf_md"], ),
    r"Fit result :",
    r'slope = %.3f +/- %.3f mmag/mm' % (1000*popt[0],1000*perr[0],),
    r'pearson r = %.3f , p-val = %.5f' % (pearsonr[0],pearsonr[1],),))
    #ax4.plot(xfit,yfit,'r-')
    ax4.text(0.05, 0.8, textstr2, transform=ax4.transAxes, fontsize=14,verticalalignment='top', bbox=props)

    df_fit_psf.loc[count] = [good_idx, index_in_summary, refcol_psf_md, popt[0]*1000, perr[0]*1000, separation_angle]  
    
    tmin = df_col["time1"].min()
    tmax = df_col["time1"].max()
    dt = tmax-tmin
    tmin = tmin-dt/10
    tmax = tmax+dt/10

    the_tmin=tmin
    the_tmax=tmax

    ax2.set_xlim(the_tmin,the_tmax)
    ax2.grid()
    #plt.xticks(rotation=45)
    ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=45, ha='right',fontsize=10)
    plt.show()
    count += 1
    #if count > 2:
    #    break
   


In [ ]:
fig,ax  = plt.subplots(figsize=(10,6))
ax.errorbar(x=df_fit_psf["color"],y=df_fit_psf["slope"],yerr=df_fit_psf["slope_err"],ecolor="k",fmt=".",color="k") 
sc = df_fit_psf.plot.scatter(x="color",y="slope",c="sep",marker= "o",s=50,ax=ax,colormap="jet")
ax.grid()
#ax.set_ylim(-5.,10)
ax.set_title("Slope vs relative psf color $\Delta C = |(G-R)_1- (G-R)_2|$")
ax.set_xlabel("relative psf color $\Delta C = |(G-R)_1- (G-R)_2|$ (mag) ")
ax.set_ylabel("$d(\Delta C)/d(PWV)$ (mmag/mm) ")
#cbar = plt.colorbar(sc)
#cbar.ax.set_xlabel('separation angle (arcsec)', rotation=0)